In [1]:
import pandas as pd

df = pd.read_csv("time.csv")

In [2]:
df.head()

,canvas_index,currencyStringFormat,date,value,link
0,1,USD%s,Dec 23,138562,https://www.hipflat.co.th/en/projects/ratchada...
1,1,USD%s,Jan 24,135913,https://www.hipflat.co.th/en/projects/ratchada...
2,1,USD%s,Feb 24,132184,https://www.hipflat.co.th/en/projects/ratchada...
3,1,USD%s,Mar 24,134525,https://www.hipflat.co.th/en/projects/ratchada...
4,1,USD%s,Apr 24,130812,https://www.hipflat.co.th/en/projects/ratchada...


In [3]:
import re
# Define function to extract condo name using regex
def extract_condo_name(url):
    match = re.search(r'projects/([a-zA-Z0-9\-]+)', url)
    if match:
        return match.group(1)
    return None

# Apply the function to the 'link' column to create the 'condo_name' column
df['condo_name'] = df['link'].apply(extract_condo_name)

In [4]:
# Define function to assign 'type' based on 'currencyStringFormat'
def assign_type(row):
    if row['currencyStringFormat'] == 'USD%s':
        return 'Price'
    elif row['currencyStringFormat'] == 'USD%s/sqm':
        return 'Price Per Space'
    return 'Other'

# Apply the function to the DataFrame to create the 'type' column
df['type'] = df.apply(assign_type, axis=1)
# Show the updated DataFrame
print(df)


        canvas_index currencyStringFormat    date   value  \
0                  1                USD%s  Dec 23  138562   
1                  1                USD%s  Jan 24  135913   
2                  1                USD%s  Feb 24  132184   
3                  1                USD%s  Mar 24  134525   
4                  1                USD%s  Apr 24  130812   
...              ...                  ...     ...     ...   
105971             1            USD%s/sqm  Aug 24      13   
105972             1            USD%s/sqm  Sep 24      14   
105973             1            USD%s/sqm  Oct 24      14   
105974             1            USD%s/sqm  Nov 24      13   
105975             1            USD%s/sqm  Dec 24      13   

                                                     link  \
0       https://www.hipflat.co.th/en/projects/ratchada...   
1       https://www.hipflat.co.th/en/projects/ratchada...   
2       https://www.hipflat.co.th/en/projects/ratchada...   
3       https://www.hip

In [5]:
def update_type(df):
    rent_count = 0
    rent_per_space_count = 0

    # Define the conditions to check
    conditions = ['currencyStringFormat', 'date', 'link', 'condo_name']

    for index, row in df.iterrows():
        # Select rows with the same conditions excluding the value column
        matching_rows = df[(df['currencyStringFormat'] == row['currencyStringFormat']) &
                           (df['date'] == row['date']) &
                           (df['link'] == row['link']) &
                           (df['condo_name'] == row['condo_name'])]

        for _, match in matching_rows.iterrows():
            if row['value'] < match['value']:
                if row['type'] in ['Price', 'Price Per Space']:
                    if row['type'] == 'Price':
                        df.at[index, 'type'] = 'Rent'
                        rent_count += 1
                    elif row['type'] == 'Price Per Space':
                        df.at[index, 'type'] = 'Rent Per Space'
                        rent_per_space_count += 1

    return df, rent_count, rent_per_space_count

df, rent_count, rent_per_space_count = update_type(df)
print(df)
print(f"Rent count: {rent_count}, Rent Per Space count: {rent_per_space_count}")

        canvas_index currencyStringFormat    date   value  \
0                  1                USD%s  Dec 23  138562   
1                  1                USD%s  Jan 24  135913   
2                  1                USD%s  Feb 24  132184   
3                  1                USD%s  Mar 24  134525   
4                  1                USD%s  Apr 24  130812   
...              ...                  ...     ...     ...   
105971             1            USD%s/sqm  Aug 24      13   
105972             1            USD%s/sqm  Sep 24      14   
105973             1            USD%s/sqm  Oct 24      14   
105974             1            USD%s/sqm  Nov 24      13   
105975             1            USD%s/sqm  Dec 24      13   

                                                     link  \
0       https://www.hipflat.co.th/en/projects/ratchada...   
1       https://www.hipflat.co.th/en/projects/ratchada...   
2       https://www.hipflat.co.th/en/projects/ratchada...   
3       https://www.hip

In [8]:
# Pivot the table: each 'date' becomes a new column with 'value' as data
pivot_df = df.pivot_table(index=['canvas_index', 'currencyStringFormat', 'link', 'condo_name'], columns='date', values='value', aggfunc='first')

# Reset index to have all columns in a flat structure
pivot_df.reset_index(inplace=True)

In [10]:
# Pivot the table: each 'date' becomes a new column with 'value' as data and include 'type'
pivot_df = df.pivot_table(index=['canvas_index', 'currencyStringFormat', 'link', 'condo_name', 'type'],
                          columns='date', values='value', aggfunc='first')

# Print the columns to check if "Dec 24" exists
print("Columns in pivoted DataFrame:", pivot_df.columns)

# Explicitly order the columns based on the required date sequence
ordered_columns = ["canvas_index", "currencyStringFormat", "link", "condo_name", "type",
                   "Dec 23", "Jan 24", "Feb 24", "Mar 24", "Apr 24", "May 24", "Jun 24",
                   "Jul 24", "Aug 24", "Sep 24", "Oct 24", "Nov 24","Dec 24"]

# Check if all columns exist in the DataFrame
missing_columns = [col for col in ordered_columns if col not in pivot_df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")
else:
    pivot_df = pivot_df[ordered_columns]

# Reset index to have all columns in a flat structure
pivot_df.reset_index(inplace=True)

# Show the updated DataFrame
print(pivot_df)
print(f"Rent count: {rent_count}, Rent Per Space count: {rent_per_space_count}")


Columns in pivoted DataFrame: Index(['Apr 24', 'Aug 24', 'Dec 23', 'Dec 24', 'Feb 24', 'Jan 24', 'Jul 24',
       'Jun 24', 'Mar 24', 'May 24', 'Nov 24', 'Oct 24', 'Sep 24'],
      dtype='object', name='date')
Missing columns: ['canvas_index', 'currencyStringFormat', 'link', 'condo_name', 'type']
date  canvas_index currencyStringFormat  \
0                1                USD%s   
1                1                USD%s   
2                1                USD%s   
3                1                USD%s   
4                1                USD%s   
...            ...                  ...   
8147             1            USD%s/sqm   
8148             1            USD%s/sqm   
8149             1            USD%s/sqm   
8150             1            USD%s/sqm   
8151             1            USD%s/sqm   

date                                               link  \
0     https://www.hipflat.co.th/en/projects/15-suite...   
1     https://www.hipflat.co.th/en/projects/15-suite...   
2     ht

In [11]:
pivot_df

date,canvas_index,currencyStringFormat,link,condo_name,type,Apr 24,Aug 24,Dec 23,Dec 24,Feb 24,Jan 24,Jul 24,Jun 24,Mar 24,May 24,Nov 24,Oct 24,Sep 24
0,1,USD%s,https://www.hipflat.co.th/en/projects/15-suite...,15-suite-buacrl,Price,313929,333262,308486,324894,333937,335024,318335,304842,332831,317803,328191,339569,344451
1,1,USD%s,https://www.hipflat.co.th/en/projects/15-suite...,15-suite-buacrl,Rent,1229,1321,1288,1319,1270,1315,1246,1225,1262,1229,1325,1358,1366
2,1,USD%s,https://www.hipflat.co.th/en/projects/15-sukhu...,15-sukhumvit-residences-htzxov,Price,313929,333262,308486,324894,333937,335024,318335,304842,332831,317803,328191,339569,344451
3,1,USD%s,https://www.hipflat.co.th/en/projects/15-sukhu...,15-sukhumvit-residences-htzxov,Rent,1229,1321,1288,1319,1270,1315,1246,1225,1262,1229,1325,1358,1366
4,1,USD%s,https://www.hipflat.co.th/en/projects/168-sukh...,168-sukhumvit-36-sxiyrw,Price,354282,392387,352736,384009,375679,371174,375329,365107,371376,372594,389375,403184,408694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8147,1,USD%s/sqm,https://www.hipflat.co.th/en/projects/zenith-p...,zenith-place-sukhumvit-42-urwhyg,Rent Per Space,19,20,19,20,19,19,19,19,19,19,20,21,21
8148,1,USD%s/sqm,https://www.hipflat.co.th/en/projects/zenith-p...,zenith-place-sukhumvit-71-rnywvv,Price Per Space,5190,5539,5287,5554,5288,5297,5263,5154,5299,5215,5543,5732,5781
8149,1,USD%s/sqm,https://www.hipflat.co.th/en/projects/zenith-p...,zenith-place-sukhumvit-71-rnywvv,Rent Per Space,19,20,19,20,19,19,19,19,19,19,20,21,21
8150,1,USD%s/sqm,https://www.hipflat.co.th/en/projects/zenithar...,zenithar-penthouse-sukhumvit-cdmwcx,Price Per Space,4423,4620,4441,4720,4476,4473,4431,4405,4504,4482,4678,4765,4789


In [12]:
# Sort the DataFrame by condo_name in ascending order
pivot_df_sorted = pivot_df.sort_values(by='condo_name', ascending=True)

In [13]:
pivot_df_sorted

date,canvas_index,currencyStringFormat,link,condo_name,type,Apr 24,Aug 24,Dec 23,Dec 24,Feb 24,Jan 24,Jul 24,Jun 24,Mar 24,May 24,Nov 24,Oct 24,Sep 24
0,1,USD%s,https://www.hipflat.co.th/en/projects/15-suite...,15-suite-buacrl,Price,313929,333262,308486,324894,333937,335024,318335,304842,332831,317803,328191,339569,344451
1,1,USD%s,https://www.hipflat.co.th/en/projects/15-suite...,15-suite-buacrl,Rent,1229,1321,1288,1319,1270,1315,1246,1225,1262,1229,1325,1358,1366
4076,1,USD%s/sqm,https://www.hipflat.co.th/en/projects/15-suite...,15-suite-buacrl,Price Per Space,4423,4620,4441,4720,4476,4473,4431,4405,4504,4482,4678,4765,4789
4077,1,USD%s/sqm,https://www.hipflat.co.th/en/projects/15-suite...,15-suite-buacrl,Rent Per Space,18,18,18,19,18,18,18,18,18,18,19,19,19
2,1,USD%s,https://www.hipflat.co.th/en/projects/15-sukhu...,15-sukhumvit-residences-htzxov,Price,313929,333262,308486,324894,333937,335024,318335,304842,332831,317803,328191,339569,344451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4072,1,USD%s,https://www.hipflat.co.th/en/projects/zenith-p...,zenith-place-sukhumvit-71-rnywvv,Price,354282,392387,352736,384009,375679,371174,375329,365107,371376,372594,389375,403184,408694
8150,1,USD%s/sqm,https://www.hipflat.co.th/en/projects/zenithar...,zenithar-penthouse-sukhumvit-cdmwcx,Price Per Space,4423,4620,4441,4720,4476,4473,4431,4405,4504,4482,4678,4765,4789
4074,1,USD%s,https://www.hipflat.co.th/en/projects/zenithar...,zenithar-penthouse-sukhumvit-cdmwcx,Price,313929,333262,308486,324894,333937,335024,318335,304842,332831,317803,328191,339569,344451
4075,1,USD%s,https://www.hipflat.co.th/en/projects/zense-re...,zense-residence-3ht1l37a1hu1ucu0947c3c2lct2n312n,Price,436,462,516,440,501,572,445,461,449,443,447,481,486


In [20]:
# Remove the 'date' and 'canvas_index' columns and group by 'condo_name'
pivot_df_grouped = pivot_df_sorted.drop(['canvas_index'], axis=1)

# Group by 'condo_name' and aggregate the values
pivot_df_grouped = pivot_df_grouped.groupby('condo_name').agg('first').reset_index()



In [14]:
# Check if all columns exist in the DataFrame
missing_columns = [col for col in ordered_columns if col not in pivot_df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")
else:
    pivot_df = pivot_df[ordered_columns]

# Sort by 'condo_name'
pivot_df = pivot_df.sort_values(by='condo_name')

# Reset index to have all columns in a flat structure
pivot_df.reset_index(inplace=True)

# Show the updated DataFrame
print(pivot_df)
print(f"Rent count: {rent_count}, Rent Per Space count: {rent_per_space_count}")


date  index  canvas_index currencyStringFormat  \
0         0             1                USD%s   
1         1             1                USD%s   
2      4076             1            USD%s/sqm   
3      4077             1            USD%s/sqm   
4         2             1                USD%s   
...     ...           ...                  ...   
8147   4072             1                USD%s   
8148   8150             1            USD%s/sqm   
8149   4074             1                USD%s   
8150   4075             1                USD%s   
8151   8151             1            USD%s/sqm   

date                                               link  \
0     https://www.hipflat.co.th/en/projects/15-suite...   
1     https://www.hipflat.co.th/en/projects/15-suite...   
2     https://www.hipflat.co.th/en/projects/15-suite...   
3     https://www.hipflat.co.th/en/projects/15-suite...   
4     https://www.hipflat.co.th/en/projects/15-sukhu...   
...                                          

In [15]:
# Save the grouped DataFrame to a CSV file
pivot_df.to_csv('time_test.csv', index=False)

# Provide feedback on the save operation
print("CSV file has been saved as 'time_test.csv'")


CSV file has been saved as 'time_test.csv'
